In [52]:
import os
import torch
import torchvision
import numpy as np

from matching import Matcher

In [47]:
data_dirpath='D:/data/mask_rcnn'
fg_iou_thresh=0.7
bg_iou_thresh=0.3

device=torch.device("cpu")
assign_targets_to_anchors=torch.load(os.path.join(data_dirpath, "assign_targets_to_anchors.pt"),map_location=device, weights_only=True)
targets=assign_targets_to_anchors['targets']
anchors=assign_targets_to_anchors['anchors']
print('targets ', [t['boxes'].shape for t in targets])
print('anchors ', [a.shape for a in anchors])

proposal_matcher = Matcher(fg_iou_thresh,bg_iou_thresh,allow_low_quality_matches=True)

targets  [torch.Size([4, 4]), torch.Size([2, 4])]
anchors  [torch.Size([185460, 4]), torch.Size([185460, 4])]


In [51]:
labels,matched_gt_boxes=[],[]
for anchors_per_image, targets_per_image in zip(anchors, targets):
    gt_boxes=targets_per_image['boxes']

    if gt_boxes.numel()==0:
        # Background image (negative example)
        device=anchors_per_image.device
        matched_gt_boxes_per_image=torch.zeros(anchors_per_image.shape, dtype=torch.float32, device=device)
        labels_per_image=torch.zeros((anchors_per_image.shape[0],), dtype=torch.float32, device=device)
    else:
        # Nx4 Mx4 => NxM
        match_quality_matrix=torchvision.ops.box_iou(gt_boxes, anchors_per_image)
        matched_idxs=proposal_matcher(match_quality_matrix)
        print('matched_idxs ', matched_idxs.shape, matched_idxs.min(), matched_idxs.max())
        # get the targets corresponding to ground truth for each proposal
        # Note: need to clamp the indices because we can have a single ground truth in the image,
        # matched_idxs can be -2, which goes out of bounds
        matched_gt_boxes_per_image=gt_boxes[matched_idxs.clamp(min=0)]

        labels_per_image=matched_idxs>=0
        labels_per_image=labels_per_image.to(dtype=torch.float32)

        # Background (negative examples)
        bg_indices=matched_idxs==proposal_matcher.BELOW_LOW_THRESHOLD
        labels_per_image[bg_indices]=0.

        # discard indices that are between thresholds
        inds_to_discard=matched_idxs==proposal_matcher.BETWEEN_THRESHOLDS
        labels_per_image[inds_to_discard]=-1.
    labels.append(labels_per_image)
    matched_gt_boxes.append(matched_gt_boxes_per_image)


matched_idxs  torch.Size([185460]) tensor(-2) tensor(3)
matched_idxs  torch.Size([185460]) tensor(-2) tensor(1)


torch.Size([185460])